In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
spark.version

'3.5.0'

In [4]:
!dir -lh ffhv_tripdata_2019-10.csv

 Volume in drive D is New Volume
 Volume Serial Number is 4CEB-F69E

 Directory of d:\GitHub Project\data-engineering-zoomcamp\05-batch\code


 Directory of d:\GitHub Project\data-engineering-zoomcamp\05-batch\code



File Not Found


In [5]:
df = spark.read \
    .option("header", "true") \
    .csv('fhv_tripdata_2019-10.csv')

In [6]:
df.schema

StructType([StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropOff_datetime', StringType(), True), StructField('PUlocationID', StringType(), True), StructField('DOlocationID', StringType(), True), StructField('SR_Flag', StringType(), True), StructField('Affiliated_base_number', StringType(), True)])

In [7]:
from pyspark.sql import types

In [8]:
# schema =  types.StructType([
#     types.StructField('hvfhs_license_num', types.StringType(), True),
#     types.StructField('dispatching_base_num', types.StringType(), True),
#     types.StructField('pickup_datetime', types.TimestampType(), True),
#     types.StructField('dropoff_datetime', types.TimestampType(), True),
#     types.StructField('PULocationID', types.IntegerType(), True),
#     types.StructField('DOLocationID', types.IntegerType(), True),
#     types.StructField('SR_Flag', types.StringType(), True)
# ])
schema =  types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropOff_datetime', types.TimestampType(), True), 
    types.StructField('PUlocationID', types.IntegerType(), True), 
    types.StructField('DOlocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True), 
    types.StructField('Affiliated_base_number', types.StringType(), True)])

In [9]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv')



In [10]:
df = df.repartition(6)

In [ ]:
df.write.mode('overwrite').parquet('fhv_tripdata/2019/10/')

In [11]:
df = spark.read.parquet('fhv_tripdata/2019/10/')

**Q3**: How many taxi trips were there on February 15?

In [12]:
from pyspark.sql import functions as F

In [13]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2019-10-15'") \
    .count()

62610

In [25]:
df.registerTempTable('fhvhv_2021_02')

In [28]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    fhvhv_2021_02
WHERE
    to_date(pickup_datetime) = '2021-02-15';
""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+




[Stage 20:==============>                                           (1 + 3) / 4]



**Q4**: Longest trip for each day

In [29]:
df.columns

['hvfhs_license_num',
 'dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag']

In [36]:
df \
    .withColumn('duration', df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2021-02-11|        75540|
| 2021-02-17|        57221|
| 2021-02-20|        44039|
| 2021-02-03|        40653|
| 2021-02-19|        37577|
+-----------+-------------+




[Stage 38:==================================================>   (187 + 4) / 200]



In [38]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 60) AS duration
FROM 
    fhvhv_2021_02
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

+-----------+-----------------+
|pickup_date|         duration|
+-----------+-----------------+
| 2021-02-11|           1259.0|
| 2021-02-17|953.6833333333333|
| 2021-02-20|733.9833333333333|
| 2021-02-03|           677.55|
| 2021-02-19|626.2833333333333|
| 2021-02-25|            583.5|
| 2021-02-18|576.8666666666667|
| 2021-02-10|569.4833333333333|
| 2021-02-21|           537.05|
| 2021-02-09|534.7833333333333|
+-----------+-----------------+




[Stage 44:================================================>     (180 + 4) / 200]



**Q5**: Most frequent `dispatching_base_num`

How many stages this spark job has?



In [44]:
spark.sql("""
SELECT
    dispatching_base_num,
    COUNT(1)
FROM 
    fhvhv_2021_02
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()


[Stage 73:>                                                         (0 + 4) / 4]



+--------------------+--------+
|dispatching_base_num|count(1)|
+--------------------+--------+
|              B02510| 3233664|
|              B02764|  965568|
|              B02872|  882689|
|              B02875|  685390|
|              B02765|  559768|
+--------------------+--------+




[Stage 74:===================================================>  (189 + 5) / 200]



In [46]:
df \
    .groupBy('dispatching_base_num') \
        .count() \
    .orderBy('count', ascending=False) \
    .limit(5) \
    .show()


[Stage 86:>                                                         (0 + 4) / 4]



+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
+--------------------+-------+




[Stage 87:===========================================>          (161 + 5) / 200]



**Q6**: Most common locations pair

In [47]:
df_zones = spark.read.parquet('zones')

In [49]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [51]:
df.columns

['hvfhs_license_num',
 'dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag']

In [50]:
df_zones.registerTempTable('zones')

In [57]:
spark.sql("""
SELECT
    CONCAT(pul.Zone, ' / ', dol.Zone) AS pu_do_pair,
    COUNT(1)
FROM 
    fhvhv_2021_02 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                      LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY 
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+--------------------+--------+
|          pu_do_pair|count(1)|
+--------------------+--------+
|East New York / E...|   45041|
|Borough Park / Bo...|   37329|
| Canarsie / Canarsie|   28026|
|Crown Heights Nor...|   25976|
|Bay Ridge / Bay R...|   17934|
+--------------------+--------+

